In [1]:
import math

import matplotlib.pyplot as plt
import numpy as np
import torch

from machine_learning import NeuralNetwork
from motor_controller import connect_arduino

In [2]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [3]:
def gradient(target: torch.Tensor, start: torch.Tensor) -> (np.ndarray, float):
    start.requires_grad = True
    current = model.forward(start)
    loss = torch.nn.functional.mse_loss(current, target)
    loss.backward(retain_graph=True)
    grad = start.grad.detach()
    start.requires_grad = False
    err = torch.mean(torch.abs((target - current) / target))
    return grad.numpy(), float(err)


In [4]:
def search(
        target: np.ndarray = np.array([600, 600], dtype=float),
        start: np.ndarray = np.array([700, 350], dtype=float),
        max_iterations: int = int(1e4),
        epsilon: float = 1e-3,
        step_size: float = 0.1,
) -> (np.ndarray, float):
    target, start = target.copy(), start.copy()
    cnt, err = 0, np.inf
    while cnt < max_iterations and err > epsilon:
        grad, err = gradient(
            torch.tensor(target, dtype=torch.float),
            torch.tensor(start, dtype=torch.float),
        )
        start -= step_size * grad
        cnt += 1
    return np.round(start, decimals=2), np.round(err, decimals=4)


In [5]:
search()

(array([832.41, 289.61]), 0.001)

In [6]:
line_segment = np.array([
    [x_position, 800]
    for x_position in np.linspace(100, 600, 101)
], dtype=float)
line_segment

array([[100., 800.],
       [105., 800.],
       [110., 800.],
       [115., 800.],
       [120., 800.],
       [125., 800.],
       [130., 800.],
       [135., 800.],
       [140., 800.],
       [145., 800.],
       [150., 800.],
       [155., 800.],
       [160., 800.],
       [165., 800.],
       [170., 800.],
       [175., 800.],
       [180., 800.],
       [185., 800.],
       [190., 800.],
       [195., 800.],
       [200., 800.],
       [205., 800.],
       [210., 800.],
       [215., 800.],
       [220., 800.],
       [225., 800.],
       [230., 800.],
       [235., 800.],
       [240., 800.],
       [245., 800.],
       [250., 800.],
       [255., 800.],
       [260., 800.],
       [265., 800.],
       [270., 800.],
       [275., 800.],
       [280., 800.],
       [285., 800.],
       [290., 800.],
       [295., 800.],
       [300., 800.],
       [305., 800.],
       [310., 800.],
       [315., 800.],
       [320., 800.],
       [325., 800.],
       [330., 800.],
       [335.,

In [22]:
circle = np.array([
    np.array([350, 700]) + 200 * np.array([math.cos(theta), math.sin(theta)])
    for theta in np.linspace(-math.pi, math.pi, 181)
], dtype=float)
circle

array([[150.        , 700.        ],
       [150.1218346 , 693.02010066],
       [150.48718995, 686.04870525],
       [151.09562093, 679.09430735],
       [151.94638625, 672.16537981],
       [153.0384494 , 665.27036447],
       [154.37047985, 658.41766184],
       [155.94085474, 651.61562088],
       [157.74766081, 644.87252884],
       [159.78869674, 638.19660113],
       [162.06147584, 631.59597133],
       [164.56322909, 625.07868132],
       [167.29090847, 618.65267138],
       [170.24119074, 612.32577064],
       [173.41048143, 606.10568744],
       [176.79491924, 600.        ],
       [180.39038077, 594.01614715],
       [184.19248549, 588.16141931],
       [188.19660113, 582.44294954],
       [192.39784928, 576.86770493],
       [196.79111138, 571.44247806],
       [201.3710349 , 566.17387873],
       [206.13203993, 561.06832591],
       [211.06832591, 556.13203993],
       [216.17387873, 551.3710349 ],
       [221.44247806, 546.79111138],
       [226.86770493, 542.39784928],
 

In [8]:
def hilbert_curve(x, y, xi, xj, yi, yj, n):
    if n <= 0:
        yield x + (xi + yi) / 2, y + (xj + yj) / 2
    else:
        yield from hilbert_curve(x, y, yi / 2, yj / 2, xi / 2, xj / 2, n - 1)
        yield from hilbert_curve(x + xi / 2, y + xj / 2, xi / 2, xj / 2, yi / 2, yj / 2, n - 1)
        yield from hilbert_curve(x + xi / 2 + yi / 2, y + xj / 2 + yj / 2, xi / 2, xj / 2, yi / 2, yj / 2, n - 1)
        yield from hilbert_curve(x + xi / 2 + yi, y + xj / 2 + yj, -yi / 2, -yj / 2, -xi / 2, -xj / 2, n - 1)


In [ ]:
# Starting point and direction vectors
x, y = 250, 600
xi, xj = 200, 0
yi, yj = 0, 200

# Generate Hilbert curve points
n = 4
points = np.array(list(hilbert_curve(x, y, xi, xj, yi, yj, n)), dtype=float)

# Plot the curve
plt.plot(points[:, 0], points[:, 1], marker=".", linewidth=1)
plt.axis("square")
plt.show()

In [ ]:
points.shape

In [ ]:
motor_steps, errors = [], []
for idx in range(len(points)):
    position, error = search(
        target=points[idx],
        start=motor_steps[-1] if len(motor_steps) > 0 else np.array([1000, 250], dtype=float),
    )
    motor_steps.append(position.copy()), errors.append(error)
    print(idx, error)
motor_steps = np.array(motor_steps, dtype=float)
motor_steps

In [ ]:
# plt.plot(motor_steps, label="Motor Input")
# plt.plot(errors, label="Errors")
plt.plot(motor_steps[:, 0], motor_steps[:, 1], label="Input - Motor", linewidth=1)
plt.plot(points[:, 0], points[:, 1], label="Output - Paper", linewidth=1)
plt.legend()
plt.axis("square")
plt.show()

In [ ]:
l_controller, r_controller = connect_arduino()

In [ ]:
for l_step, r_step in motor_steps:
    r_controller.set_position(r_step)
    l_controller.set_position(l_step)


In [ ]:
l_controller.set_position(0), r_controller.set_position(0)